#Logging metrics to an AML run 

In [2]:
import azureml 
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.get_details())

{'id': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/PipelinesUsabilityStudy/providers/Microsoft.MachineLearningServices/workspaces/shwinneworkshop', 'name': 'shwinneworkshop', 'location': 'eastus2', 'type': 'Microsoft.MachineLearningServices/workspaces', 'tags': {}, 'sku': 'Enterprise', 'workspaceid': '2c99da6f-e7fc-4070-b1f7-204807455abf', 'description': '', 'friendlyName': '', 'creationTime': '2020-01-13T20:47:53.0433365+00:00', 'containerRegistry': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourceGroups/PipelinesUsabilityStudy/providers/Microsoft.ContainerRegistry/registries/shwinneworks933cbd9b', 'keyVault': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/pipelinesusabilitystudy/providers/microsoft.keyvault/vaults/shwinneworksho8870415227', 'applicationInsights': '/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/pipelinesusabilitystudy/providers/microsoft.insights/components/shwinneworksho2947248937', 'identit

In [3]:
from azureml.core import Experiment

#create new aml experiment 
experiment = Experiment(ws, 'logging_run_metrics')

In [34]:
%%writefile train.py


from azureml.core import Run

run = Run.get_context() 

#Log a metric value. This is logged as a scalar to the 'Metrics' tab
run.log('Accuracy', 4)

#log an image to a run. This is of type Image and is logged under the 'Outputs + logs' tab and can be found in the 'logs' folder
run.log_image('food', path='./breadpudding.jpg', plot=None, description='food is life')

#log list of values to a run. This is logged as a 'Vector' and can be found under the 'Metrics' tab *the chart is off and the index looks strange. 
#x axis is 'Index'
my_list1 = ['a', 'b', 'c']
my_list2 = [1, 2, 3]
run.log_list('Letters', value=my_list1, description='some letters')
run.log_list('Numbers', value=my_list2, description='some numbers')

#log a row to a run. This is logged as type 'Table' and is found under the 'Metrics' tab
citrus = ['orange', 'lemon', 'lime'] 
sizes = [ 10, 7, 3] #y axis

#x axis is fruit
#y axis is sizes 
    
for index in range(len(citrus)):
    run.log_row("citrus", fruit = citrus[index], size=sizes[index])


#log a table to a run. This is logged as type 'Table' and is found under the 'Metrics' tab and under the 'Table' option. First value(keys) is the column name  

fruits = dict([('apples', '5'), ('pears', '9'), ('oranges', '7'), ('bananas', '4'), ('plums', '2') ])
run.log_table('test table', fruits, description='testing table')

#log confusion matrix to an AML run. This is logged as a confusion_matrix type and can be found under the 'Outputs + logs' tab and can be found in the 'logs' folder
# More info - https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html 

from sklearn.metrics import confusion_matrix
from azureml.core import Run

run = Run.get_context()

y_true = ["cat", "dog", "cat", "cat", "dog", "fish"]
y_pred = ["dog", "dog", "cat", "cat", "dog", "cat"]
matrix = confusion_matrix(y_true, y_pred, labels=["dog", "fish", "cat"])

run.log_confusion_matrix(name='confusion_matrix', 
                         value={
                           "schema_type": "confusion_matrix",
                           "schema_version": "v1",
                           "data": { 
                               "class_labels": ["dog", "fish", "cat"], 
                               "matrix": matrix.tolist() 
                           }
                         })





Overwriting train.py


In [35]:
from azureml.core.environment import CondaDependencies
from azureml.core import Environment
from azureml.core import ScriptRunConfig


#Instead of managing the setup of the environment yourself, you can ask the system to build a new conda environment for you. 
#The environment is built once, and will be reused in subsequent executions as long as the conda dependencies remain unchanged.

myenv = Environment(name="myenv")
conda_dep = CondaDependencies()
#to log the sklearn confusion matrix, you must add the scikit-learn package
conda_dep.add_pip_package("scikit-learn")
myenv.python.conda_dependencies=conda_dep

myenv.python.user_managed_dependencies = False

#To submit a run, create a run configuration that combines the script file and environment, and pass it to Experiment.submit. 
#In this example, the script is submitted to local computer, but you can specify other compute targets such as remote clusters as well.

runconfig = ScriptRunConfig(source_directory=".", script="train.py")
runconfig.run_config.target = "local"
runconfig.run_config.environment = myenv
run = experiment.submit(config=runconfig)

run.wait_for_completion(show_output=True)

RunId: logging_run_metrics_1584977574_46017b4e
Web View: https://ml.azure.com/experiments/logging_run_metrics/runs/logging_run_metrics_1584977574_46017b4e?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/pipelinesusabilitystudy/workspaces/shwinneworkshop

Streaming azureml-logs/70_driver_log.txt

Starting the daemon thread to refresh tokens in background for process with pid = 24406
Entering Run History Context Manager.
Preparing to call script [ train.py ] with arguments: []
After variable expansion, calling script [ train.py ] with arguments: []



The experiment completed successfully. Finalizing run...
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 24406
Cleaning up all outstanding Run operations, waiting 300.0 seconds
2 items cleaning up...
Cleanup took 0.38193607330322266 seconds

Execution Summary
RunId: logging_run_metrics_1584977574_46017b4e
Web View: https://ml.a

{'runId': 'logging_run_metrics_1584977574_46017b4e',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-03-23T15:33:07.645253Z',
 'endTimeUtc': '2020-03-23T15:33:12.313636Z',
 'properties': {'_azureml.ComputeTargetType': 'local',
  'ContentSnapshotId': 'da723c60-f4fe-4c9c-8bca-e65daa1a132f',
  'azureml.git.repository_uri': 'https://github.com/fastai/course-v3.git',
  'mlflow.source.git.repoURL': 'https://github.com/fastai/course-v3.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '19a04b283c18445059d8ad745b5dc28a0f40a053',
  'mlflow.source.git.commit': '19a04b283c18445059d8ad745b5dc28a0f40a053',
  'azureml.git.dirty': 'True'},
 'inputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'useAbsolutePath': False,
  'arguments': [],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'local',
  'dataReferences': {},
  'data': {},
  'jobName': None,
  'maxRunDurationSe